In [1]:
import pandas as pd

pd.set_option("display.width", 500)
pd.set_option("display.max_colwidth", 200)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

%matplotlib inline

In [19]:
gastos = pd.read_csv("../data/processed/federal/despesas/gastos diretos/ufc_201301_201701.csv", decimal=',')

gastos['data'] = pd.to_datetime(gastos.data, format="%Y-%m-%d")

gastos['nome_favorecido'] = gastos.nome_favorecido.str.strip()

In [20]:
fcpc = pd.read_csv("../data/processed/fcpc/fisica.csv", parse_dates=[4], date_parser=lambda d: pd.datetime.strptime(d, '%Y-%m-%d'))

### Quem aparece nos dois datasets?

In [21]:
nos_dois = pd.merge(gastos, fcpc, right_on="nome", left_on="nome_favorecido", suffixes=("_gastos", "_fcpc")).nome.unique()

nos_dois.shape

(3150,)

### Recebido por ano/mes

In [77]:
gastos_ano_mes = gastos[gastos.nome_favorecido.isin(nos_dois)]\
.groupby(['nome_favorecido', gastos.data.dt.year, gastos.data.dt.month, "elemento_despesa"]).valor.sum()\
.unstack()

In [78]:
fcpc_ano_mes = fcpc[fcpc.nome.isin(nos_dois)]\
.groupby(['nome', fcpc.data.dt.year, fcpc.data.dt.month]).valor.sum()\
.rename("valor_fcpc")

In [94]:
recebeu_mais_1 = pd.concat([fcpc_ano_mes, gastos_ano_mes], axis=1)
recebeu_mais_1.index.set_names(['nome', 'ano', 'mes'], inplace=True)

### Quem recebeu simultaneamente de duas formas num mesmo ano/mes

In [95]:
recebeu_mais_1_simultaneamente_nomes = recebeu_mais_1[recebeu_mais_1.notnull().sum(axis=1) > 1].index.get_level_values("nome")

In [96]:
recebeu_mais_1_simultaneamente = recebeu_mais_1[recebeu_mais_1.index.get_level_values("nome").isin(recebeu_mais_1_simultaneamente_nomes)]

### Analisando a distribuição do total recebido

In [124]:
pct_recebimentos = recebeu_mais_1_simultaneamente.notnull().groupby(level=0).sum().apply(lambda x: x / x.sum(), axis=1)

In [141]:
distribuidos = pct_recebimentos[((pct_recebimentos > 0.2) & (pct_recebimentos < 0.5)).sum(axis=1) > 1]

### Desses, qual o top 5?

In [155]:
recebeu_mais_1_simultaneamente[recebeu_mais_1_simultaneamente.index.get_level_values("nome").isin(distribuidos.index.get_level_values("nome"))]\
.groupby(level=0).sum().sum(axis=1).nlargest(5)

nome
EMANUEL EDUARDO DA SILVA OLIVEIRA   51541.38
LUCAS RAFAEL PINHEIRO LOPES         40148.41
LUCAS KUCHLA                        30785.24
CAROLINA MAIA ALMEIDA BEZERRA       30616.46
EDUARDO DA SILVA MAIA               26681.70
dtype: float64

In [160]:
recebeu_mais_1[recebeu_mais_1.index.get_level_values("nome").isin(["EDUARDO DA SILVA MAIA"])]\
[['valor_fcpc', 'Outros Serviços de Terceiros - Pessoa Física', 'Auxílio Financeiro a Estudantes']]

valor_fcpc  Outros Serviços de Terceiros - Pessoa Física  Auxílio Financeiro a Estudantes
nome                  ano  mes                                                                                           
EDUARDO DA SILVA MAIA 2013 7           nan                                        800.00                              nan
                           8           nan                                        400.00                              nan
                           9           nan                                        400.00                              nan
                           10          nan                                        400.00                              nan
                           11          nan                                        400.00                              nan
                           12          nan                                        800.00                              nan
                      2014 4           nan                                        400.00                              nan
                           5           nan                                        400.00                              nan
                           6           nan                                        400.00                              nan
                           7         95.24                                        800.00                              nan
                           9           nan                                        400.00                              nan
                           10          nan                                        800.00                              nan
                           12          nan                                        800.00                              nan
                      2015 4           nan                                        400.00                              nan
                           5           nan                                        400.00                           511.90
                           6           nan                                        400.00                           515.63
                           7           nan                                        400.00                           515.63
                           8           nan                                        400.00                           515.63
                           9           nan                                        400.00                           511.90
                           10          nan                                        400.00                           515.63
                           11          nan                                        400.00                           511.90
                           12       190.48                                        800.00                          1031.26
                      2016 2           nan                                           nan                          1023.80
                           4        190.48                                           nan                           525.10
                           5           nan                                           nan                           929.27
                           6           nan                                           nan                           925.10
                           7           nan                                           nan                           929.27
                           8           nan                                           nan                           929.27
                           9        380.96                                           nan                           925.10
                           10          nan                                           nan                           929.27
                           11          nan                                           nan                           925.10
      